In [1]:
import geopandas as gpd
import pandas as pd
import vaex

/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df_iptu_distrito = vaex.open('data/IPTU-1995-2022-agrupados-por-distrito.hdf5')
df_iptu_subprefeitura = vaex.open('data/IPTU-1995-2022-agrupados-por-subprefeitura.hdf5')
df_iptu_od = vaex.open('data/IPTU-1995-2022-agrupados-por-od.hdf5')

In [3]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')
gdf_distritos['area'] = gdf_distritos.area
gdf_distritos.geometry = gdf_distritos.simplify(tolerance=100)
gdf_distritos.to_crs(epsg=4674, inplace=True)

gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017', crs='EPSG:31983')
gdf_od = gdf_od[gdf_od.od_municip == '36']
gdf_od.set_crs(epsg=31983, inplace=True)
gdf_od['area'] = gdf_od.area
gdf_od.geometry = gdf_od.simplify(tolerance=10)
gdf_od.to_crs(epsg=4674, inplace=True)

gdf_subprefeitura = gpd.read_file('zip://data/SIRGAS_GPKG_subprefeitura.zip!SIRGAS_GPKG_subprefeitura.gpkg')
gdf_subprefeitura['area'] = gdf_subprefeitura.area
gdf_subprefeitura.geometry = gdf_subprefeitura.simplify(tolerance=100)
gdf_subprefeitura.to_crs(epsg=4674, inplace=True)

In [32]:
ano = [2002, 2022]
atributo = "Percentual de Uso Residencial"

In [35]:
gdf_agregacao = gdf_distritos.astype({'ds_codigo': 'int'})\
    .merge(df_iptu_distrito[df_iptu_distrito.ano >= ano[0] & df_iptu_distrito.ano <= ano[-1]].to_pandas_df(), \
        left_on='ds_codigo', right_on='distrito')\
            [["ds_codigo", "ds_nome", atributo, 'geometry', 'ano', 'Quantidade de Unidades']]

In [41]:
(df_iptu_distrito.ano >= ano[0]) & (df_iptu_distrito.ano <= ano[1])

Expression = ((ano >= 2002) & (ano <= 2022))
Length: 2,716 dtype: bool (expression)
--------------------------------------
   0  False
   1  False
   2  False
   3  False
   4  False
    ...    
2711   True
2712   True
2713   True
2714   True
2715   True

In [34]:
gdf_agregacao

,ds_codigo,ds_nome,Percentual de Uso Residencial,geometry,ano,Quantidade de Unidades
0,63,PIRITUBA,0.802808,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1995,30876
1,63,PIRITUBA,0.796576,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1996,31136
2,63,PIRITUBA,0.790158,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1997,31387
3,63,PIRITUBA,0.803951,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1998,32479
4,63,PIRITUBA,0.800241,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1999,32702
...,...,...,...,...,...,...
2683,37,ITAQUERA,0.835931,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2018,43181
2684,37,ITAQUERA,0.835703,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2019,43683
2685,37,ITAQUERA,0.835090,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2020,44164
2686,37,ITAQUERA,0.816535,"POLYGON ((-46.43236 -23.52627, -46.42933 -23.5...",2021,45811


In [24]:
gdf_diff = gdf_agregacao.pivot(index='ds_codigo', columns='ano', values=atributo)

In [43]:
diff = (gdf_var[ano[-1]] - gdf_var[ano[0]])

In [48]:
gdf_distritos[atributo] = diff

In [49]:
gdf_distritos

,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,geometry,area,Percentual de Uso Residencial
0,63,PIRITUBA-JARAGUA,02,PIR,PIRITUBA,"POLYGON ((-46.71217 -23.45452, -46.71544 -23.4...",1.712166e+07,NaN
1,95,PIRITUBA-JARAGUA,02,SDO,SAO DOMINGOS,"POLYGON ((-46.76387 -23.47407, -46.76027 -23.4...",9.886971e+06,0.062185
2,42,PIRITUBA-JARAGUA,02,JAR,JARAGUA,"POLYGON ((-46.77026 -23.45337, -46.76936 -23.4...",2.832976e+07,-0.005145
3,11,FREGUESIA-BRASILANDIA,03,BRL,BRASILANDIA,"POLYGON ((-46.71437 -23.46231, -46.71490 -23.4...",2.107930e+07,-0.173466
4,29,FREGUESIA-BRASILANDIA,03,FRE,FREGUESIA DO O,"POLYGON ((-46.68976 -23.50903, -46.70579 -23.5...",1.101703e+07,0.008886
...,...,...,...,...,...,...,...,...
91,44,SAO MIGUEL,23,JDH,JARDIM HELENA,"POLYGON ((-46.44027 -23.47986, -46.43500 -23.4...",9.143813e+06,0.027230
92,24,ITAQUERA,27,CLD,CIDADE LIDER,"POLYGON ((-46.51685 -23.55514, -46.51140 -23.5...",1.035511e+07,0.004018
93,57,ITAQUERA,27,PQC,PARQUE DO CARMO,"POLYGON ((-46.44461 -23.59272, -46.45600 -23.5...",1.545145e+07,0.041697
94,47,ITAQUERA,27,JBO,JOSE BONIFACIO,"POLYGON ((-46.42933 -23.53341, -46.42308 -23.5...",1.457084e+07,0.006954
